In [37]:
import finnhub
import os
from pydantic import BaseModel
from dotenv import load_dotenv
from typing import Optional
import time

load_dotenv()

class StockPriceResponse(BaseModel):
    current_price: float
    previous_close: float
    change: Optional[float] = None
    change_percent: Optional[float] = None
    high: Optional[float] = None
    low: Optional[float] = None
    open: Optional[float] = None

def get_stock_price(symbol: str) -> StockPriceResponse:
    """
    Get stock price using Finnhub API
    
    Args:
        symbol: Stock ticker symbol (e.g., 'AAPL', 'GOOGL')
    
    Returns:
        StockPriceResponse with current and previous close prices
    
    Raises:
        Exception: If API call fails or returns invalid data
    """
    # Initialize Finnhub client with API key from .env
    api_key = os.getenv('FINHUB_API_KEY')
    if not api_key:
        raise ValueError("FINHUB_API_KEY not found in .env file")
    
    finnhub_client = finnhub.Client(api_key=api_key)
    
    try:
        # Get quote data for the symbol
        quote = finnhub_client.quote(symbol.upper())
        
        # Check if we got valid data
        if not quote or quote.get('c') is None:
            raise ValueError(f"No data available for symbol: {symbol}")
        
        # Extract the data
        current_price = quote['c']  # Current price
        previous_close = quote['pc']  # Previous close
        
        # Create response with additional data if available
        return StockPriceResponse(
            current_price=current_price,
            previous_close=previous_close,
            change=quote.get('d'),  # Change ($)
            change_percent=quote.get('dp'),  # Change (%)
            high=quote.get('h'),  # Day high
            low=quote.get('l'),  # Day low
            open=quote.get('o')  # Day open
        )
        
    except Exception as e:
        raise Exception(f"Failed to fetch stock price for {symbol}: {str(e)}")


In [45]:
get_stock_price("LCID")

StockPriceResponse(current_price=18.41, previous_close=16.16, change=2.25, change_percent=13.9233, high=18.735, low=16.3, open=16.34)

In [46]:
from agents import Agent, FileSearchTool, Runner, WebSearchTool, function_tool

@function_tool
async def fetch_stock_price(symbol: str) -> StockPriceResponse:
    return get_stock_price(symbol)

In [50]:
from agents import Agent

stock_research_agent = Agent(
  name="Stock Research Agent",
  instructions="""
  You are a Stock Market Researcher. Your job is to research the current news around a specific stock and determine what may have caused recent price movements in the past 24 hours specifically. You have access to be able to retreive the stock price and research via the internet using your tools.

  The stock that you are being asked to research has moved within the past 24 hours which has caused an automation to trigger your research. Make sure you have an explanation for the movement in the stock price.

  You should use the fetch_stock_price tool to check current stock price, previous close, and other relevant data. Use the WebSearchTool to find recent news articles or information about the stock.

  Output: Your final_output should be a the price, an short analysis of the price change and short summary of your findings that is no longer than 160 characters.
  """,
  tools=[
    fetch_stock_price,
    WebSearchTool()
  ],
  model="gpt-4.1"
)

In [51]:
from agents import Runner

async def main():
    result = await Runner.run(stock_research_agent, "LCID")
    print(result.final_output)

In [52]:

import asyncio

if __name__ == "__main__":
    await main()

OPENAI_API_KEY is not set, skipping trace export


Lucid Group Inc. (LCID) shares rose 13.92% to $18.41, up $2.25 from the previous close of $16.16.

This surge follows Uber's announcement of a $300 million investment in Lucid and a commitment to purchase over 20,000 autonomous-ready Gravity SUVs, marking a significant advancement in Lucid's autonomous vehicle initiatives. ([reuters.com](https://www.reuters.com/business/autos-transportation/uber-invest-300-million-ev-maker-lucid-part-robotaxi-deal-2025-07-17/?utm_source=openai))

## Stock market information for Lucid Group Inc (LCID)
- Lucid Group Inc is a equity in the USA market.
- The price is 18.41 USD currently with a change of 2.25 USD (0.14%) from the previous close.
- The latest open price was 16.3 USD and the intraday volume is 33386947.
- The intraday high is 18.73 USD and the intraday low is 16.15 USD.
- The latest trade time is Saturday, September 6, 00:15:00 UTC.
 


OPENAI_API_KEY is not set, skipping trace export
